In [1]:
import pandas as pd 
import numpy as np 

In [8]:
cd Desktop/

/Users/siandavies/Desktop


In [9]:
!ls

categorical.csv   numerical.csv     numerical.csv.zip target.csv


In [10]:
numerical=pd.read_csv('numerical.csv')

In [11]:
categorical=pd.read_csv('categorical.csv')

In [12]:
target=pd.read_csv('target.csv')

In [13]:
numerical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95412 entries, 0 to 95411
Columns: 315 entries, TCODE to CLUSTER2
dtypes: float64(9), int64(306)
memory usage: 229.3 MB


In [14]:
categorical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95412 entries, 0 to 95411
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   STATE         95412 non-null  object
 1   CLUSTER       95412 non-null  int64 
 2   HOMEOWNR      95412 non-null  object
 3   GENDER        95412 non-null  object
 4   DATASRCE      95412 non-null  int64 
 5   RFA_2R        95412 non-null  object
 6   RFA_2A        95412 non-null  object
 7   GEOCODE2      95412 non-null  object
 8   DOMAIN_A      95412 non-null  object
 9   DOMAIN_B      95412 non-null  int64 
 10  ODATEW_YR     95412 non-null  int64 
 11  ODATEW_MM     95412 non-null  int64 
 12  DOB_YR        95412 non-null  int64 
 13  DOB_MM        95412 non-null  int64 
 14  MINRDATE_YR   95412 non-null  int64 
 15  MINRDATE_MM   95412 non-null  int64 
 16  MAXRDATE_YR   95412 non-null  int64 
 17  MAXRDATE_MM   95412 non-null  int64 
 18  LASTDATE_YR   95412 non-null  int64 
 19  LAST

In [15]:
target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95412 entries, 0 to 95411
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   TARGET_B  95412 non-null  int64  
 1   TARGET_D  95412 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 1.5 MB


In [16]:
data=pd.concat([numerical,categorical,target],axis=1)

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95412 entries, 0 to 95411
Columns: 339 entries, TCODE to TARGET_D
dtypes: float64(10), int64(322), object(7)
memory usage: 246.8+ MB


In [18]:
data['TARGET_B'].value_counts()

0    90569
1     4843
Name: TARGET_B, dtype: int64

# imbalance management - downsampling 

In [19]:
category_0=data[data['TARGET_B']==0].sample(len(data[data['TARGET_B']==1]))

In [20]:
category_1=data[data['TARGET_B']==1]

In [21]:
data=pd.concat([category_0,category_1],axis=0)

In [22]:
data=data.reset_index(drop=True)

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9686 entries, 0 to 9685
Columns: 339 entries, TCODE to TARGET_D
dtypes: float64(10), int64(322), object(7)
memory usage: 25.1+ MB


In [24]:
data.head()

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TARGET_B,TARGET_D
0,2,61.611649,1,1,4,0,28,23,41,12,...,89,3,94,9,96,1,86,11,0,0.0
1,0,61.611649,7,9,0,0,24,26,24,3,...,95,12,95,11,95,12,94,11,0,0.0
2,1,76.000000,2,7,1,0,27,27,34,4,...,87,3,95,2,95,9,86,9,0,0.0
3,3,88.000000,5,9,0,0,41,6,54,7,...,90,11,95,12,95,12,88,9,0,0.0
4,0,60.000000,1,9,0,0,26,28,45,11,...,92,11,94,11,95,10,92,3,0,0.0


In [25]:
#set dependent variable 
y=data['TARGET_B']
X=data.drop(['TARGET_B'],axis=1)

In [26]:
X_num=X.select_dtypes(np.number)
X_cat=X.select_dtypes(object)

In [27]:
X_cat.head()

,STATE,HOMEOWNR,GENDER,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A
0,other,U,F,L,E,D,R
1,NC,U,F,L,F,C,R
2,other,H,M,L,E,C,T
3,FL,H,F,L,F,B,C
4,FL,U,M,L,E,A,U


In [28]:
X_cat.shape

(9686, 7)

In [29]:
from sklearn.preprocessing import OneHotEncoder

In [30]:
encoder=OneHotEncoder(drop='first').fit(X_cat)

In [31]:
X_cat_en=encoder.transform(X_cat).toarray() # lost categorical feature names

In [42]:
X_cat_en_cols=encoder.get_feature_names_out(X_cat.columns)# restore feature names 
#older version 'get_feature_names'

In [43]:
X_cat_en_cols

array(['STATE_FL', 'STATE_GA', 'STATE_IL', 'STATE_IN', 'STATE_MI',
       'STATE_MO', 'STATE_NC', 'STATE_TX', 'STATE_WA', 'STATE_WI',
       'STATE_other', 'HOMEOWNR_U', 'GENDER_M', 'GENDER_other',
       'RFA_2A_E', 'RFA_2A_F', 'RFA_2A_G', 'GEOCODE2_B', 'GEOCODE2_C',
       'GEOCODE2_D', 'DOMAIN_A_R', 'DOMAIN_A_S', 'DOMAIN_A_T',
       'DOMAIN_A_U'], dtype=object)

In [44]:
X_cat_en_df=pd.DataFrame(X_cat_en, columns=X_cat_en_cols) # restored categorical names 

In [45]:
X_full=pd.concat([X_cat_en_df,X_num],axis=1)# retained feature names 

In [47]:
X_full.head(1)

,STATE_FL,STATE_GA,STATE_IL,STATE_IN,STATE_MI,STATE_MO,STATE_NC,STATE_TX,STATE_WA,STATE_WI,...,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TARGET_D
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2,89,3,94,9,96,1,86,11,0.0


In [34]:
from sklearn.model_selection import train_test_split 

In [48]:
X_train,X_test,y_train,y_test=train_test_split(X_full,y,test_size=0.3,random_state=40)

In [36]:
from sklearn.ensemble import RandomForestClassifier

In [49]:
clf=RandomForestClassifier(n_estimators=200,max_depth=2, random_state=40)
clf.fit(X_train,y_train)
print(clf.score(X_test,y_test))

0.9390915347556779


In [38]:
X_full.shape

(9686, 355)

# with ensemble you should use more than one scoring metric 

In [39]:
from sklearn.model_selection import cross_val_score

In [50]:
cross_val_scores=cross_val_score(clf,X_train,y_train)

In [51]:
cross_val_scores  # there are more parameters we havent set up 

array([0.92551622, 0.94616519, 0.94616519, 0.96238938, 0.94985251])

# Feature selection / scoring with RF 

In [53]:
feature_importances=clf.feature_importances_

In [54]:
feature_names = X_full.columns

In [55]:
forest_importances = pd.Series(feature_importances, index=feature_names)

In [56]:
forest_importances.sort_values(ascending=False)

TARGET_D    0.113624
LASTGIFT    0.069838
AVGGIFT     0.063221
MINRAMNT    0.049156
RFA_2F      0.046409
              ...   
ETH7        0.000000
HHD10       0.000000
OCC12       0.000000
OCC13       0.000000
STATE_FL    0.000000
Length: 355, dtype: float64

In [59]:
with pd.option_context('display.max_rows', None) :
    print(forest_importances.sort_values(ascending=False))

TARGET_D        0.113624
LASTGIFT        0.069838
AVGGIFT         0.063221
MINRAMNT        0.049156
RFA_2F          0.046409
CARDGIFT        0.045724
MAXRAMNT        0.043340
RFA_2A_G        0.037687
NGIFTALL        0.035077
LASTDATE_YR     0.024061
ETH1            0.016853
NUMPRM12        0.014083
ETH2            0.012402
CARDPM12        0.012286
IC1             0.011538
ETHC5           0.010882
CARDPROM        0.010339
HHD9            0.009998
HV1             0.009808
IC5             0.009195
HVP3            0.008409
ETHC4           0.008329
HV2             0.008087
DMA             0.007960
FIRSTDATE_YR    0.007531
HVP4            0.006598
IC3             0.006586
IC4             0.006490
HVP5            0.006198
HHN2            0.006136
HC21            0.005967
HHAS3           0.005881
AGE901          0.005864
CONTROLN        0.005599
AGEC6           0.005511
CLUSTER2        0.005439
NUMPROM         0.005437
RAMNTALL        0.005209
ETHC2           0.005046
IC2             0.005004


In [ ]:
X_cat_en_cols=encoder.get_feature_names_out(X_cat.columns)

In [ ]:
X_cat_en_df=pd.DataFrame(X_cat_en, columns=X_cat_en_cols)

In [ ]:
X_cat_en_df.head(1)

In [ ]:
X_full=pd.concat([X_cat_en_df,X_num],axis=1)

In [ ]:
X_full.head(1)

In [ ]:
clf.feature_importances_

In [ ]:
feature_names = X_full.columns

In [ ]:
feature_names

In [ ]:
import time
start_time = time.time()
importances = clf.feature_importances_
std = np.std([clf.feature_importances_ for tree in clf.estimators_], axis=0)
elapsed_time = time.time() - start_time

In [ ]:
print(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None) :
    print(forest_importances.sort_values(ascending=False))

In [ ]:
import matplotlib.pyplot as plt 

In [ ]:
forest_importances = pd.Series(importances, index=feature_names)

fig, ax = plt.subplots(figsize=(50,30))
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()